# Reconstruction and analysis of B-cell lineage trees from single cell data using Immcantation


![](assets/dowser-tutorial-cover.png)



Human B cells play a fundamental role in the adaptive immune response to infection and vaccination, as well as the pathology of allergies and many autoimmune diseases. Central to all of these processes is the fact that B cells are an evolutionary system, and undergo rapid somatic hypermutation and antigen-driven selection as part of the adaptive immune response. The similarities between this B cell response and evolution by natural selection have made phylogenetic methods a powerful means of characterizing important processes, such as immunological memory formation. Recent methodological work has led to the development of phylogenetic methods that adjust for the unique features of B cell evolution. Further, advances in single cell sequencing can now provide an unprecedented resolution of information, including linked heavy and light chain data, as well as the associated transcriptional states of individual B cells. In this tutorial, we show how single cell information can be integrated into B cell phylogenetic analysis using the Immcantation suite (Immcantation.org).

**This tutorial covers:**

Beginning with processed single cell RNA-seq (scRNA-seq) + BCR data from 10X Genomics, we will show:

- how cell type annotations can be associated with BCR sequences,
- how clonal clusters can be identified, and 
- how B cell phylogenetic trees can be built and visualized using these data sources.

## Resources

- You can email [immcantation@googlegroups.com](mailto:immcantation@googlegroups.com) with any questions or issues.
- Documentation: http://immcantation.org
- Source code and bug reports: https://bitbucket.org/kleinstein/immcantation
- Docker/Singularity container for this lab: https://hub.docker.com/r/immcantation/lab

## How to use the notebook

Jupyter Notebook documentation: https://jupyter-notebook.readthedocs.io/en/stable/

**Ctrl+Enter** will run the code in the selected cell and **Shift+Enter** will run the code and move to the following cell.

## Inside this container

This container comes with software and example data that is ready to use. The commands `versions report` and `builds report` show the versions and dates respectively of the tools and data.

### Software versions
Use this command to list the software versions

In [ ]:
%%bash
versions report

### Build versions
Use this command to list the date and changesets used during the image build.

In [ ]:
%%bash
builds report

### Example data used in the tutorial

- `../data/BCR.data.tsv`: B-Cell Receptor Data. Adaptive Immune Receptor Repertoire (AIRR) tsv BCRs already aligned to IMGT V, D, and J genes. To learn more visit  https://immcantation.readthedocs.io/en/stable/tutorials/tutorials.html


- `../data/GEX.data.rds`: Gene Expression Data. This file contains a Seurat object with RNA-seq data already processed and annotated. For examples visit https://satijalab.org/seurat/articles/pbmc3k_tutorial.html


## Outline of tutorial

1. B cell phylogenetics background.

1. Combining gene expression and BCR sequences.

1. Identifying clonal clusters, reconstruct germlines.

1. Building and visualizing trees.

1. Tree analysis, detecting ongoing evolution.




## B cells underlie both immune function and pathology

![](assets/dowser-tutorial/bcells.png)

## BCRs are first produced by random recombination


## Each B cell has a single type of receptor


## B cell affinity maturation



## Adaptive Immune Receptor Repertoire (AIRR) sequencing



## B cell phylogenetic inference


## Trees link sources of B cell diversity


## Read in data to R session


In [ ]:
# R for Python
# Enable use of R magic to run R code
import rpy2.rinterface
%load_ext rpy2.ipython

In [ ]:
%%R
suppressPackageStartupMessages(library(alakazam))
suppressPackageStartupMessages(library(Seurat))

# Read BCR data
bcr_db <- readChangeoDb("../data/BCR.data.tsv")

# Read GEX data
gex_db <- readRDS("../data/GEX.data.rds")


## What’s in the box?



In [ ]:
%%R
library(Seurat)
# Object summary
print(gex_db)

In [ ]:
%%R
# Cell type annotations
head(Idents(gex_db),1)

In [ ]:
%%R
suppressPackageStartupMessages(library(dplyr))
# Object summary
head(bcr_db,1)

In [ ]:
%%R
# check out select columns
head(select(bcr_db, cell_id, v_call, j_call, sample, day),1)

## Add BCR data to Seurat object


In [ ]:
%%R
# Make cell IDs in BCR match those in Seurat Object
bcr_db$cell_id_unique = paste0(bcr_db$sample, "_", bcr_db$cell_id)
bcr_db$cell_id_unique = gsub("-1","", bcr_db$cell_id_unique)
bcr_db$cell_id_unique[1]

In [ ]:
%%R
Cells(gex_db)[1]

In [ ]:
%%R
# match index from BCR to GEX information
match.index = match(Cells(gex_db), bcr_db$cell_id_unique)
# What proportion of cells don’t have BCRs?
mean(is.na(match.index))

In [ ]:
%%R
# Just to double check..
mean(Cells(gex_db) == bcr_db$cell_id_unique[match.index],na.rm=TRUE)


### Do cells annotated as B cells actually have BCRs? Do non-B cells express BCRs?

In [ ]:
%%R
# label whether BCR found in cell
gex_db$contains_bcr = !is.na(match.index)
# List of cells with BCRs
highlighted.cells = Cells(gex_db)[which(gex_db$contains_bcr)]
# Plot UMAP with BCR-containing cells
DimPlot(object = gex_db, reduction = "umap",
   cells.highlight = highlighted.cells, label =
   TRUE, cols="gray", pt.size = 1.0, label.size=8,
   label.box=TRUE) + NoLegend()



## Add GEX data to BCR object


In [ ]:
%%R
# Match indexes from GEX to BCR data
# Different from BCR to GEX!
match.index = match(bcr_db$cell_id_unique, Cells(gex_db))
# What proportion of BCRs don’t have GEX information?
mean(is.na(match.index))

In [ ]:
%%R
# Add annotations to BCR data
cell.annotation = as.character(Idents(gex_db))
bcr_db$gex_annotation= unlist(lapply(match.index,
function(x){ifelse(is.na(x),NA, cell.annotation[x])}))

In [ ]:
%%R
# Add UMAP coordinates to BCR data
umap1 = gex_db@reductions$umap@cell.embeddings[,1]
umap2 = gex_db@reductions$umap@cell.embeddings[,2]
bcr_db$gex_umap1= unlist(lapply(match.index,
function(x){ifelse(is.na(x),NA, umap1[x])}))
bcr_db$gex_umap2= unlist(lapply(match.index,
function(x){ifelse(is.na(x),NA, umap2[x])}))

In [ ]:
%%R
# Remove cells that didn’t match
bcr_db = filter(bcr_db, !is.na(gex_annotation))

### Ensure information transferred from Seurat object

In [ ]:
%%R
suppressPackageStartupMessages(library(ggplot2))
# Set up color palette for annotations
col_anno = c(
"GC B"="dodgerblue2",
"PB"="firebrick2",
"ABC"="seagreen",
"Naive B"="darkgoldenrod2",
"RMB"="plum2",
"Germline"="black")
# Plot UMAP from bcr_db
ggplot(bcr_db) +
geom_point(aes(x = gex_umap1, y = gex_umap2,
color = gex_annotation)) +
scale_colour_manual(values=col_anno) +
theme_bw()


## Add GEX data to BCR object



In [ ]:
%%R
# Plot isotype on UMAP
ggplot(bcr_db) +
geom_point(aes(x=gex_umap1,
y = gex_umap2,
color = isotype)) +
theme_bw()

## Identifying clonal clusters




## Picking a threshold using shazam




In [ ]:
%%R
suppressPackageStartupMessages(library(shazam))
# Find threshold using heavy chains
dist_ham <- distToNearest(filter(bcr_db,
locus=="IGH"))
output <- findThreshold(dist_ham$dist_nearest)
threshold <- output@threshold
# Plot distance to each nearest neighbor
plotDensityThreshold(output)


## Performing clustering using scoper




In [ ]:
%%R
suppressPackageStartupMessages(library(scoper))
# Assign clonal clusters
results <- hierarchicalClones(dist_ham,
threshold=threshold)
results_db <- as.data.frame(results)
# get clone sizes using dplyr functions
clone_sizes <- countClones(results_db)
# Plot cells per clone
ggplot(clone_sizes, aes(x=seq_count))+
geom_bar() + theme_bw() +
xlab("Sequences per clone")



## Reconstruct germlines using dowser

Note: If you opted for the native installation, you can obtain reference germlines from IMGT with:

```
git clone https://bitbucket.org/kleinstein/immcantation
immcantation/scripts/fetch_imgtdb.sh
```
 



In [ ]:
%%R
suppressPackageStartupMessages(library(dowser))

# read in IMGT data if downloaded on your own (above)
# and update `dir` to use the path to your `human/vdj` folder
# references = readIMGT(dir = "human/vdj/")

# read in IMGT data if using in Docker image
references = readIMGT(dir = "/usr/local/share/germlines/imgt/human/vdj")
# Reconstruct germlines
results_db = createGermlines(results_db, references)
# Check output column
results_db$germline_alignment_d_mask[1]


## Formatting clones with dowser



In [ ]:
%%R
# Make clone objects with aligned, processed sequences
# collapse identical sequences unless differ by trait
# add up duplicate_count column for collapsed sequences
# store day, isotype, gex_annotation
# discard clones with < 5 distinct sequences
clones = formatClones(results_db,
   traits = c("day", "isotype", "gex_annotation"),
   num_fields=c("duplicate_count"), minseq=5)
clones


## Constructing trees



## Tree building with dowser




In [ ]:
%%R
# Two options for maximum parsimony trees
trees = getTrees(clones)
trees = getTrees(clones, build="dnapars",
exec="/usr/local/bin/dnapars")
# Two options for standard maximum likelihood trees
trees = getTrees(clones, build="pml", sub_model="GTR")
trees = getTrees(clones, build="dnaml",
exec="/usr/local/bin/dnaml")
# B cell specific maximum likelihood with IgPhyML
trees = getTrees(clones, build="igphyml",
exec="/usr/local/share/igphyml/src/igphyml", nproc=2)
trees


## Plotting trees with dowser and ggtree

All tree building methods are plotted using the same method in dowser.



In [ ]:
%%R
# Plot all trees
plots = plotTrees(trees, tips="isotype", tipsize=2)

In [ ]:
%%R
# Plot the largest tree
plots[[1]]

In [ ]:
%%R
# Save PDF of all trees
treesToPDF(plots, file="results/final_data_trees.pdf", nrow=2, ncol=2)

## More elaborate tree plots




In [ ]:
%%R
suppressPackageStartupMessages(library(ggtree))
# Scale branches to mutations rather than mutations/site
trees = scaleBranches(trees)
# Make fancy tree plot of second largest tree
plotTrees(trees, scale=5)[[2]] +
geom_tippoint(aes(colour=gex_annotation, size=day)) +
geom_tiplab(aes(label=isotype), offset=0.002) +
scale_colour_manual(values = col_anno)

## Reconstruct intermediate sequences




In [ ]:
%%R
suppressPackageStartupMessages(library(ggtree))
# Collapse nodes with identical sequences
trees = collapseNodes(trees)
# node_nums=TRUE labels each internal node
p = plotTrees(trees, node_nums=TRUE, labelsize=6, scale=5)[[2]] +
   geom_tippoint(aes(colour=gex_annotation, size=day)) + 
   geom_tiplab(aes(label=isotype), offset=0.002) + 
   scale_colour_manual(values = col_anno)
# Get sequence at node 26
getSeq(trees, clone=1018, node=26)

## Are lineages measurably evolving?




## Detecting measurable evolution




## Correlation tests with dowser




In [ ]:
%%R
# Correlation test
trees = correlationTest(trees, time="day")
# remove trees with one timepoint, order by p value
trees = filter(trees, !is.na(p))
trees = trees[order(trees$p),]
# Fancy tree plots
p = plotTrees(trees)
p = lapply(p, function(x)
   x + 
  geom_tippoint(aes(fill=day),shape=21, size=3)+
  scale_fill_distiller(palette="RdYlBu"))

# Save
treesToPDF(p,file="results/time_data_trees.pdf")

select(trees, clone_id, slope, correlation, p)

# References

## B cell phylo 

Hoehn, K. B. et al. (2016) The diversity and molecular evolution of B-cell receptors during infection. MBE. https://doi.org/10.1093/molbev/msw015

Hoehn, K. B. et al. (2019) Repertoire-wide phylogenetic models of B cell molecular evolution reveal evolutionary signatures of aging and vaccination. PNAS 201906020.

Hoehn, K. B. et al. (2020) Phylogenetic analysis of migration, differentiation, and class switching in B cells.
bioRxiv. https://doi.org/10.1101/2020.05.30.124446

Hoehn, K. B. et al. (2021) Human B cell lineages engaged by germinal centers following influenza vaccination are measurably evolving. bioRxiv. https://doi.org/10.1101/2021.01.06.425648


## BCR analysis

Gupta,N.T. et al. (2017) Hierarchical clustering can identify b cell clones
with high confidence in ig repertoire sequencing data. The Journal of
Immunology, 1601850.

Gupta,N.T. et al. (2015) Change-o: A toolkit for analyzing large-scale b cell
immunoglobulin repertoire sequencing data. Bioinformatics, 31, 3356–3358.

Nouri,N. and Kleinstein,S.H. (2018a) A spectral clustering-based method
for identifying clones from high-throughput b cell repertoire sequencing data.
Bioinformatics, 34, i341–i349.

Nouri,N. and Kleinstein,S.H. (2018b) Optimized threshold inference for
partitioning of clones from high-throughput b cell repertoire sequencing
data. Frontiers in immunology, 9.

Stern,J.N. et al. (2014) B cells populating the multiple sclerosis brain
mature in the draining cervical lymph nodes. Science translational medicine,
6, 248ra107–248ra107.


Vander Heiden,J.A. et al. (2017) Dysregulation of b cell repertoire
formation in myasthenia gravis patients revealed through deep sequencing.
The Journal of Immunology, 1601415.

Yaari,G. et al. (2012) Quantifying selection in high-throughput
immunoglobulin sequencing data sets. Nucleic acids research, 40,
e134–e134.

Yaari,G. et al. (2013) Models of somatic hypermutation targeting and
substitution based on synonymous mutations from high-throughput
immunoglobulin sequencing data. Frontiers in immunology, 4, 358.
